In [4]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt

# loading data
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [12]:
# info on the dataset 
print(f"x_train.shape: {x_train.shape}") # (50000, 32, 32, 3) *3: color channels
print(f"y_train.shape: {y_train.shape}") #(50000, 1)

print(f"x_test.shape: {x_test.shape}") # (10000, 32, 32, 3)
print(f"y_test.shape: {y_test.shape}") # (10000, 1)


x_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000, 1)
x_test.shape: (10000, 32, 32, 3)
y_test.shape: (10000, 1)


In [ ]:
def preprocess(x_train, y_train, x_test, y_test):

    #  from 1-256 to 0-1
    x_train, x_test = x_train / 255.0, x_test / 255.0
    
    # flatten the label values
    y_train, y_test = y_train.flatten(), y_test.flatten()

    return x_train, y_train, x_test, y_test